In [8]:
import pandas as pd

In [9]:
results_df = pd.read_csv('./deseq2_feature_ranking.csv')
gene_targets = pd.read_csv('./gene_targets.csv')

/var/folders/c_/rs13jmts3nbf9xfm917mh9p80000gn/T/ipykernel_57068/193109733.py:2: DtypeWarning: Columns (236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_targets = pd.read_csv('./gene_targets.csv')


In [10]:
results_df['miRNA_rank'] = results_df['padj'].rank()
miRNA_ranks = results_df['miRNA_rank'].to_dict()

In [11]:
total_miRNA = len(results_df)

In [12]:
def calculate_gene_score(row, miRNA_ranks, total_miRNA):
    total_interactions = 0
    score = 0
    for miRNA in row.index[4:]:  # Assuming the miRNAs start from the 5th column
        if pd.notna(row[miRNA]) and row[miRNA] in miRNA_ranks:
            rank = miRNA_ranks[row[miRNA]]
            score += (1 - (rank / total_miRNA))
            total_interactions += 1
    if total_interactions > 0:
        return score / total_interactions
    else:
        return 0

In [14]:
gene_targets['gene_score'] = gene_targets.apply(calculate_gene_score, axis=1, miRNA_ranks=miRNA_ranks, total_miRNA=total_miRNA)

# Rank genes based on their scores
gene_targets['gene_rank'] = gene_targets['gene_score'].rank(ascending=False)
ranked_genes = gene_targets.sort_values(by='gene_rank')

# Save ranked genes to a CSV file
ranked_genes.to_csv('ranked_genes.csv', index=False)

# Display top ranked genes
print(ranked_genes.head(10))
print(f'ranked_genes columns: {ranked_genes.columns}')

      Gene Symbol  p-value  FDR  Odd ratio  Number of interactions  \
7627        MKNK2   0.0211  1.0      0.981                     195   
2613         CNBP   0.2690  1.0      0.991                     195   
13095    TMEM167A   0.5930  1.0      0.997                     120   
11481      SH3TC2   0.0965  1.0      0.981                     120   
4318       FAM83F   0.5930  1.0      0.997                     120   
7348          MAZ   0.5930  1.0      0.997                     120   
6590       KLHL21   0.5930  1.0      0.997                     120   
14830      ZNF566   1.0000  1.0      1.050                     120   
5789     HSD17B12   0.9920  1.0      1.030                     120   
6572      KLHDC10   0.9190  1.0      1.010                     120   

            microRNA 1       microRNA 2       microRNA 3       microRNA 4  \
7627   hsa-miR-125b-5p   hsa-miR-335-5p   hsa-miR-128-3p   hsa-miR-101-3p   
2613    hsa-miR-877-3p   hsa-miR-186-5p   hsa-miR-26a-5p    hsa-let-7e-5p  